In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
from sklearn.utils import shuffle
train = shuffle(train)

In [ ]:
features = [c for c in train.columns if c not in ("target","row_id")]
target="target"
x_test=test[features]
y = train[target]
X = train[features]
numerical_features = X.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
#Jsut for testing
X_train = X
y_train =y

In [ ]:
from sklearn.preprocessing import RobustScaler
RobustScaler_transformer = RobustScaler().fit(X_train.values)

In [ ]:
X_trainRobustScaler = RobustScaler_transformer.transform(X_train.values)
#X_validRobustScaler = RobustScaler_transformer.transform(X_valid.values)
X_testRobustScaler = RobustScaler_transformer.transform(x_test.values)

In [ ]:
X_train =pd.DataFrame(
    X_trainRobustScaler,
    columns=list(numerical_features))

In [ ]:
x_test =pd.DataFrame(
    X_testRobustScaler,
    columns=list(numerical_features))

### Import Deep Learning Libraries

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import keras
import tensorflow

import tensorflow.keras
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import numpy as np

from keras.layers.convolutional import Conv1D, MaxPooling1D


import keras.utils
from keras import utils as np_utils

from keras.utils import np_utils


from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Convolution1D, Flatten, LeakyReLU
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import SpatialDropout1D, MaxPooling1D, Bidirectional, GRU, concatenate


# import necessary tools and models 
import seaborn as sns
import matplotlib.pyplot as plt 

import sklearn.model_selection as cv
from sklearn.model_selection import cross_val_score
import numpy as np

In [ ]:
X_train.shape[1]

In [ ]:
y_train

### Compile and Train Neural Network

In [ ]:
X_train.shape[1]

In [ ]:
DL_model = keras.models.Sequential([
#    keras.layers.BatchNormalization(),
    #keras.layers.Dense(512, activation="relu",input_shape=[X_train.shape[1]]),
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=2,activation="selu", input_shape=(X_train.shape[1],1)),
    keras.layers.GaussianNoise(0.01),
    keras.layers.Dropout(0.25),
    keras.layers.Conv1D(filters=32, kernel_size=4,strides=2, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.GaussianNoise(0.01),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    #keras.layers.AveragePooling1D(pool_size=2),

    keras.layers.Dense(256, activation="selu"),
    keras.layers.GaussianNoise(0.01),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="selu"),
    keras.layers.GaussianNoise(0.01),
    keras.layers.Dropout(0.25),
#     keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation="softmax")])

In [ ]:
DL_model.summary()

In [ ]:
DL_model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
my_callbacks = [
    EarlyStopping(monitor='val_loss', 
                  min_delta=0, 
                  patience=250,
                  restore_best_weights=True)
]

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf
#https://stackoverflow.com/questions/63279168/valueerror-input-0-of-layer-sequential-is-incompatible-with-the-layer-expect
X_train = tf.expand_dims(X_train, axis=-1)

In [ ]:
history = DL_model.fit(X_train, y_train, validation_split=0.2,
             shuffle=True,
             epochs=2500,
             batch_size=256,callbacks=my_callbacks)

### Plot the loss curve

In [ ]:
plt.figure(221)


plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.xticks(np.arange(0, 100, 10))
plt.rcParams['figure.figsize'] = (25,5)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.gcf().set_size_inches(15,5)
plt.show()

### Predict on Training set and save to csv

In [ ]:
x_test = tf.expand_dims(x_test, axis=-1)

In [ ]:
test_preds = DL_model.predict(x_test,
                          batch_size=64)

In [ ]:
test_preds = np.argmax(test_preds, axis = 1)

In [ ]:
test_preds_decoded_inversed = le.inverse_transform(test_preds)

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
sub['target'] = test_preds_decoded_inversed

In [ ]:
sub.groupby(['target']).count()

In [ ]:
sub.to_csv('submission.csv', index=False)